### LIBRARY IMPORTS

In [4]:
import os
import random
import warnings

# Components for building the layered CNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Dropout, Flatten, Dense

# The ImageDataGenerator will carry out the flow of image data from storage to memory
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Set the base directory for the dataset
# Assume the dataset is present and the folder structure matches what is expected
src = 'Dataset/PetImages/'

# Import the function for splitting the dataset into the training set and the testing set
warnings.filterwarnings("ignore")
from dataset_utilities import train_test_split

### PARTITION DATASET INTO TRAINING SET AND TESTING SET 

In [2]:
# Create separate folders for storing the training samples and the test samples
#if not os.path.isdir(src + 'Train/'):

# Partition the dataset
print("Partitioning the dataset into the training set and testing set...")
train_test_split(src)

Partitioning the dataset into the training set and testing set...


### BUILD AND COMPILE THE CNN

In [5]:
# Define hyperparameters
FILTER_SIZE = 3                         # This is the sliding window that will scan an image and create a feature set
NUM_FILTERS = 32                        # We will use a total of 32 filters
INPUT_SIZE  = 64                        # Compress image dimensions to 32 x 32 (may lose some data)
MAXPOOL_SIZE = 2                        # 2 x 2 max pooling size halves the input dimensions
BATCH_SIZE = 16                         # Use 16 training samples per batch
STEPS_PER_EPOCH = 20000//BATCH_SIZE     # Number of iterations per epoch; the '//' operator divides, but truncates the decimal
EPOCHS = 10                             # Use 10 epochs

# Start with base sequential layer model
model = Sequential()

# Add first 2D convolutional layer; this layer reads the actual image files
model.add(Conv2D(NUM_FILTERS, (FILTER_SIZE, FILTER_SIZE), input_shape = (INPUT_SIZE, INPUT_SIZE, 3), activation = 'relu'))

# Add first subsampling layer using max pooling
model.add(AveragePooling2D(pool_size = (MAXPOOL_SIZE, MAXPOOL_SIZE)))

# Add second 2D convolutional layer; this layer reads the subsampled feature map from the first convolutional layer
model.add(Conv2D(NUM_FILTERS, (FILTER_SIZE, FILTER_SIZE), activation = 'relu'))

# Add second subsampling layer, also using max pooling
model.add(AveragePooling2D(pool_size = (MAXPOOL_SIZE, MAXPOOL_SIZE)))

# Flatten the multidmensional vector received from the second subsampling layer into a one-dimensional vector
model.add(Flatten())

# Add a dense fully connected layer with 128 nodes
model.add(Dense(units = 128, activation = 'relu'))

# Add dropout layer to help reduce overfitting
model.add(Dropout(0.5))

# Add second fully connected layer, with only a single node and using a sigmoid activation function
model.add(Dense(units = 1, activation = 'sigmoid'))

# Compile model using adam optimizer and binary cross-entropy loss function
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### TRAINING

In [6]:
# Create ImageDataGenerator to load batches of images at a time into memory
training_data_generator = ImageDataGenerator(rescale = 1./255)

# Create batch-loaded training set
training_set = training_data_generator.flow_from_directory(src + 'Train/',
                                                target_size = (INPUT_SIZE, INPUT_SIZE),
                                                batch_size = BATCH_SIZE,
                                                class_mode = 'binary')

# Train the model
print("Training the CNN...")
model.fit_generator(training_set, steps_per_epoch = STEPS_PER_EPOCH, epochs = EPOCHS, verbose=1)

Found 19997 images belonging to 2 classes.
Training the CNN...
Epoch 1/10
1250/1250 [==============================] - 83s 66ms/step - loss: 0.6520 - accuracy: 0.6188
Epoch 2/10
1250/1250 [==============================] - 82s 66ms/step - loss: 0.5846 - accuracy: 0.6963
Epoch 3/10
1250/1250 [==============================] - 82s 65ms/step - loss: 0.5216 - accuracy: 0.7450
Epoch 4/10
1250/1250 [==============================] - 83s 66ms/step - loss: 0.4740 - accuracy: 0.7733
Epoch 5/10
1250/1250 [==============================] - 82s 65ms/step - loss: 0.4347 - accuracy: 0.8023
Epoch 6/10
1250/1250 [==============================] - 82s 66ms/step - loss: 0.3935 - accuracy: 0.8231
Epoch 7/10
1250/1250 [==============================] - 83s 66ms/step - loss: 0.3564 - accuracy: 0.8431
Epoch 8/10
1250/1250 [==============================] - 83s 66ms/step - loss: 0.3111 - accuracy: 0.8653
Epoch 9/10
1250/1250 [==============================] - 83s 66ms/step - loss: 0.2787 - accuracy: 0.8838
E

### TESTING

In [7]:
# Create ImageDataGenerator to load batches of images at a time into memory
testing_data_generator = ImageDataGenerator(rescale = 1./255)

# Create batch-loaded testing set
test_set = testing_data_generator.flow_from_directory(src + 'Test/',
                                             target_size = (INPUT_SIZE, INPUT_SIZE),
                                             batch_size = BATCH_SIZE,
                                             class_mode = 'binary')

# Test the model
print("Testing the CNN...")
score = model.evaluate_generator(test_set, steps=100)

Found 5000 images belonging to 2 classes.
Testing the CNN...


### RESULT

In [8]:
# Display the results
print("RESULTS:")
for idx, metric in enumerate(model.metrics_names):
    print("{}: {}".format(metric, score[idx]))

RESULTS:
loss: 0.5438082218170166
accuracy: 0.7850000262260437


This model has below parameters:

FILTER_SIZE = 3                         
NUM_FILTERS = 32                        
INPUT_SIZE  = 64                        
MAXPOOL_SIZE = 2                        
BATCH_SIZE = 16                         
STEPS_PER_EPOCH = 20000//BATCH_SIZE 
EPOCHS = 10

The model used average pooling layers instead of the max-pooling subsampling layers and has 2 convolution layers.

The accuracy has gone down from 79% (original model) to 78% and loss went up from 50% to 54%.

Above CNN shows accuracy: 78.5%
Loss: 54.38% 

The model performs poorly than the original CNN model.